## Pyvis, Gephi

In [3]:
import sys
sys.path.append("..")
from rcn_py import visual

In [26]:
visual.pyvis_network_by_datafile('Stefan Verhoeven', "/Users/jennifer/RCSNA/data", "/Users/jennifer/RCSNA/visualization")

/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [4]:
visual.gephi_network_by_datafile('Stefan Verhoeven', "/Users/jennifer/RCSNA/data", "/Users/jennifer/RCSNA/visualization")

/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

D3 tbd.